# The Warehouse Problem

The Warehouse Problem is a well-know optimization case found in many textbooks.
The problem consists, given a set of candidate warehouse *locations* and a set of *stores*
to decide which warehouse to open and which warehouse will server which store.

## Input Data

Data is provided as follows:

* For each warehouse, we require a tuple (name, capacity, fixed-cost), where _name_ is the unique name of the warehouse, _capacity_ is the maximum number of stores it can supply and _fixed_cost_ is the cost incurred by opening the warehouse.
* For each couple (warehouse, store) a _supply_cost_ which estimates the cost of supplying this store by this warehouse.

A compact way of representing the data is a Python _dictionary_: for each warehouse tuple, list all supply costs for all stores.

## Business Decisions

The problem consists in deciding which warehouse will be open and for each store, by which warehouse it will be supplied.

## Business Constraints

Decisions must satisfy the following (simplified) business constraints:

1. Unicity: each store is supplied by one unique warehouse
2. A store can only be supplied by an _open_ warehouse
3. Capacity: the number of stores supplied by a warehouse must be less than its capacity

## Business Objective

The goal is to minimize the total cost incurred by the decisions, which is made of two costs:

* The *Total Opening Cost* is the sum of opening costs ranging over all open warehouses.
* The *Total Supply Cost* is the sum of supply costs for all the chosen (warehouse, store) pairs


To begin with, let's define a small warehouse dataset:

In [ ]:
WAREHOUSES = {
    ('Bonn', 1, 20): [20, 28, 74, 2, 46, 42, 1, 10, 93, 47],
    ('Bordeaux', 4, 35): [24, 27, 97, 55, 96, 22, 5, 73, 35, 65],
    ('London', 2, 32): [11, 82, 71, 73, 59, 29, 73, 13, 63, 55],
    ('Paris', 1, 37): [25, 83, 96, 69, 83, 67, 59, 43, 85, 71],
    ('Rome', 2, 25): [30, 74, 70, 61, 54, 59, 56, 96, 46, 95],
    ('Brussels', 2, 28): [30, 74, 70, 61, 34, 59, 56, 96, 46, 95]
}

print(WAREHOUSES)
print("#candidate warehouses=%d" % len(WAREHOUSES))
print("#stores to supply=%d" % len(list(WAREHOUSES.values())[0]))

Each key in the dictionary is a tuple with three fields: the name (a string), the capacity (a positive integer) and the fixed cost (also positive).
Each value is a list of length 10 (the number of stores to supply). the k^th element of the list being the supply cost from the warehouse to the k^th store. For example,

* London warehouse can supply at most two stores
* London warehouse has an opening cost of 30
* The cost incurred by supplying the first store from London is 11

## The math model

## Solving with DOcplex

Now let's solve this problem with DOcplex.
First, we define named tuples to store information relative to warehouses and stores in a clear and convenient manner.

In [ ]:
from collections import namedtuple

class TWareHouse(namedtuple("TWarehouse1", ["id", "capacity", "fixed_cost"])):
    def __str__(self):
        return self.id

class TStore(namedtuple("TStore1", ["id"])):
    def __str__(self):
        return 'store_%d' % self.id

Second, we compute the set of warehouse and store tuples. The use of *itervalues* from the *six* module is a technicality to ensure portability across Python 2 and Python3.
We chose to model stores by an integer range from 1 to nb_stores for convenience.

In [ ]:
from six import itervalues

nb_stores = 0 if not WAREHOUSES else len(next(itervalues(WAREHOUSES)))
warehouses = [TWareHouse(*wrow) for wrow in WAREHOUSES.keys()]
# we'll use the warehouses dictionary as a two-entry dictionary
supply_costs = WAREHOUSES
# we count stores from 1 to NSTORES included for convenience
stores = [TStore(idx) for idx in range(1, nb_stores + 1)]

## The model
First we need one instance of model to store our modeling artefacts

In [ ]:
try:
    import docplex.mp
except:
    !pip install docplex

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

Enter your IBM Decision Optimization on cloud credentials:

In [ ]:
url = "ENTER YOUR URL HERE" 
key = "ENTER YOUR KEY HERE"

In [ ]:
from docplex.mp.model import Model
warehouse_model = Model()

### Defining decision variables

In DOcplex, decision variables are related to objects of the business model. In our case, we have two decisions to make: which warehouse is open and, for each store, from which warehouse is it supplied.

First, we create one binary (yes/no) decision variable for each warehouse: the variable will be equal to 1 if and only if the warehouse is open.
We use the **binary_var_dict** method of the model object to create a dictionary from warehouses to variables.
The first argument states that keys of the dcitionary will be our warehouse objects;
the second argument is a a simple string *'open', that is used to generate names for variables, by suffixing 'open' with the string representation of warehouse objects (in other terms the output of the **str()** Python function.
This is the reason why we redefined the method **__str__** method of class TWarehouse in the beginning.


In [ ]:
open_vars = warehouse_model.binary_var_dict(keys=warehouses, name='open')

Second, we define one binary variable for each __pair__ or (warehouse, store). This is done using the method
**binary_var_matrix**; in this case keys to variables are all (warehouse, store) pairs.

The naming scheme applies to both components of pairs,for example the variable deciding whether the London warehouse supplies store 1 will be named supply_London_store_1.

In [ ]:
supply_vars = warehouse_model.binary_var_matrix(warehouses, stores, 'supply')

At this step, we can check how many variables we have defined. as we have 5 warehouses and 10 stores, we expect to have defined 5\*10 + 5 = 55 variables.

In [ ]:
warehouse_model.print_information()

As expected, we have not defined any constraints yet.

## Defining constraints

The first constraint states that each store is supplied by exactly one warehouse. In other terms, the sum of supply variables for a given store, ranging over all warehouses, must be equal to 1. 
Printing model information, we check that this code defines 10 constraints

In [ ]:
for s in stores:
    warehouse_model.add_constraint(warehouse_model.sum(supply_vars[w, s] for w in warehouses) == 1)
warehouse_model.print_information()

The second constraints states that a store can be suppplied only by an open warehouse. To model this, we use a little trick of logic, converting this logical implication (w supplies s) => w is open into an inequality between binary variables, as in:

In [ ]:
for s in stores:
    for w in warehouses: 
        warehouse_model.add_constraint(supply_vars[w, s] <= open_vars[w])

whenever a supply var from warehouse __w__ equals one, its open variable will also be equal to one. Conversely, when the open variable is zero, all supply variable for this warehouse will be zero.
This constraint does not prevent having an open warehouse supplying no stores.
Though this could be taken care of by adding an extra constraint, this is not necessary
as such cases will be automatically eliminated by searching for the minimal cost (see the objective section).


The third constraint states the capacity limitation on each warehouse. Note the overloading of the logical operator **<=** to express the constraint.

In [ ]:
for w in warehouses:
    warehouse_model.add_constraint(warehouse_model.sum(supply_vars[w, s] for s in stores) <= w.capacity)

At this point we can summarize the variables and constraint we have defined in the model.

In [ ]:
warehouse_model.print_information()

## Defining the Objective

The objective is to minimize the total costs. There are two costs:

* the opening cost which is incurred each time a warehouse is open
* the supply cost which is incurred by the assignments of warehouses to stores

We define two linear expressions to model these two costs and state that our objective is to minimize the sum.

In [ ]:
total_opening_cost = warehouse_model.sum(open_vars[w] * w.fixed_cost for w in warehouses)
total_opening_cost.name = "Total Opening Cost"

def _get_supply_cost(warehouse, store):
    ''' A nested function to return the supply costs from a warehouse and a store'''
    return supply_costs[warehouse][store.id - 1]

total_supply_cost = warehouse_model.sum([supply_vars[w,s] * _get_supply_cost(w, s) for w in warehouses for s in stores])
total_supply_cost.name = "Total Supply Cost"

warehouse_model.minimize(total_opening_cost + total_supply_cost)

The model is now complete and we can solve it and get the final optimal objective

## Solving the model

In [ ]:
ok = warehouse_model.solve(url=url, key=key)
assert ok
obj = warehouse_model.objective_value
print("optimal objective is %g" % obj)

But there's more: we can precise the value of those two costs, by evaluating the value of the two expressions.
Here we see that the opening cost is 140 and the supply cost is 293.

In [ ]:
opening_cost_obj = total_opening_cost.solution_value
supply_cost_obj = total_supply_cost.solution_value
print("total opening cost=%g" % opening_cost_obj)
print("total supply cost=%g" % supply_cost_obj)

# store results for later on...
results=[]
results.append((opening_cost_obj, supply_cost_obj))

## Displaying results

We can leverage the graphic toolkit __maptplotlib__ to display the results. First, let's draw a pie chart of opening cost vs. supply costs to clearly see the respective impact of both costs.

In [ ]:
if env.has_matplotlib:
    import matplotlib.pyplot as plt
    %matplotlib inline

def display_costs(costs, labels, colors):
    if env.has_matplotlib:
        plt.axis("equal")
        plt.pie(costs, labels=labels, colors=colors, autopct="%1.1f%%")
        plt.show()
    else:
        print("warning: no display")
    
display_costs(costs=[opening_cost_obj, supply_cost_obj], 
              labels=["Opening Costs", "Supply Costs"], 
              colors=["gold", "lightBlue"])

We can also display the breakdown of supply costs per warehouse.
First, compute the sum of supply cost values for each warehouse: we need to sum the actual supply cost from w to s
for those pairs (w, s) whose variable is true (here we test the value to be >=0.9, testing equality to 1 would not
be robust because of numerical precision issues)

In [ ]:
supply_cost_by_warehouse = [ sum(_get_supply_cost(w,s) for s in stores if supply_vars[w,s].solution_value >= 0.9) for w in warehouses]
wh_labels = [w.id for w in warehouses]
display_costs(supply_cost_by_warehouse, wh_labels, None)

## Exploring the Pareto frontier

remember we solved the model by minimizing the __sum__ of the two costs, getting an optimal solution
with total cost of 383, with opening costs = 120 and supply costs = 263.

In some cases, it might be intersting to wonder what is the absolute minimum of any of these two costs: what
if we'd like to minimize opening costs __first__, and __then__ the supply cost (of course keeping the best opeing cost we obtained in first phase.

This is very easy to achieve with DOcplex, using the "solve\_lexicographic" method on the model.
This method takes two arguments: the first one is a list of expressions, the second one a list of
__senses__ (Minimize or Maximize), the default being to Minimize each expression.

In this section we will explore hat happens when we minimize opening costs __then__ supply costs and conversely
when we minimize supply costs and __then__ opening costs.

#### Minimizing total opening cost and then total supply cost

In [ ]:
opening_first_then_supply = [ total_opening_cost, total_supply_cost]
warehouse_model.solve_lexicographic(goals=opening_first_then_supply)

opening1 = total_opening_cost.solution_value
supply1 = total_supply_cost.solution_value
results.append( (opening1, supply1))

From this we can see that the model has sucessfully solved twice and that the absolute minimum of opening cost is 120,
and for this value, the minimum supply cost is 263.
Remember that in our first attempt we minimized the combined sum and obtained (120,263, but now we know 120 is the best we can achieve. The supply cost is now 352 , for a total combined cost of 472 which is indeed greater than the value of 433 we found when optimizing the combined sum.

#### Minimizing total supply cost and then total opening cost

Now let's do the reverse: minimize supply cost and then opening cost. What if the goal of minimizing total supply cost supersedes the second goal of optimizing total opening cost?
The code is straightforward:

In [ ]:
supply_first_then_opening = [ total_supply_cost, total_opening_cost]
warehouse_model.solve_lexicographic(goals=supply_first_then_opening)
opening2 = total_opening_cost.solution_value
supply2 = total_supply_cost.solution_value

results.append( (opening2, supply2))

Here, we obtain (152, 288), supply costs is down from 352 to 288 but opening cost raises from 120 to 150.
We can check that the combined sum is 288+152 = 440, which is grater than the optimal of 433 we found at the beginning.


### Pareto Diagram

We can plot these three points on a (opening, supply) plane:

In [ ]:
def display_pareto(res):
    if env.has_matplotlib:
        plt.cla()
        plt.xlabel('Opening Costs')
        plt.ylabel('Supply Costs')
        colors = ['g', 'r', 'b']
        markers = ['o', '<', '>']
        nb_res = len(res)
        pts = []
        for i, r in enumerate(res):
            opening, supply = r
            p = plt.scatter(opening, supply, color=colors[i%3], s=50+10*i, marker=markers[i%3])
            pts.append(p)
        plt.legend(pts,
               ('Sum', 'Opening_first', 'Supply_first'),
               scatterpoints=1,
               loc='best',
               ncol=3,
               fontsize=8)
        plt.show()
    else:
        print("Warning: no display")

display_pareto(results)